# Image Segmentation

## Overview

### Learning Objectives

* Gain familiarity with the different categories of segmentation methods available in ITK
* Understand the difference between region growing methods and how their parameters change their behavior
* Understand the influence of terms in level set segmentation methods

### Segmentation method categories

![Segmentation categories](data/segmentation-categories.png)

### Region Growing Methods

The basic approach of a *region growing algorithm* is to **start from a seed region (typically one or more
pixels)** that are considered to be inside the object to be segmented. The **pixels neighboring this
region are evaluated to determine if they should also be considered part of the object**. If so, **they are
added to the region and the process continues as long as new pixels are added to the region**. Region
growing algorithms vary depending on the criteria used to decide whether a pixel should be included
in the region or not, the type connectivity used to determine neighbors, and the strategy used to visit
neighboring pixels.

Example region growing methods available in ITK:

- ConfidenceConnected
- ConnectedThreshold
- IsolatedConnected

#### Confidence Connected

![Confidence Connected](data/confidence-connected.png)

#### Connected Threshold

![Connected Threshold](data/connected-threshold.png)

#### Isolated Connected

![Isolated Connected](data/isolated-connected.png)

### Level Set Segmentation Methods

### Level Set Concept

![Level set concept](data/level-set-concept.png)



#### Level Set Evolution

![Level Set evolution](data/level-set-evolution.png)

#### Level Set Function

$\phi_{t} = - \alpha  \mathbf{A}(\mathbf{x})\cdot\nabla\phi - \beta   P(\mathbf{x})\mid\nabla\phi\mid + \gamma Z(\mathbf{x})\kappa\mid\nabla\phi\mid$

- $\mathbf{A}(\mathbf{x})$: **Advection**
- $P(\mathbf{x})$: **Propagation**
- $Z(\mathbf{x})$: **Spatial modifier for curvature *K***

- Track the evolution of contours and surfaces by computing the update to the solution $\phi$ of the PDE
- Contour (initial solution) is initialized by the user
- Evolve $\phi$ until
  - It fits the form of the segment you are trying to fit (shape detection)
  - It segments the features you want (segmentation)
- Evolution is uni-directional (inwards or outwards)

For more information, see:

- The *Segmentation* chapter of Book 2 of the ITK Software Guide, including the sections on *Region Growing* and *Level Set Segmentation*.

## Tutorial

In [ ]:
import numpy as np
import itk
from itkwidgets import view

We can monitor when a filter gets called by registering a command that gets called when [ProgressEvents](https://itk.org/Doxygen/html/classitk_1_1ProgressEvent.html) occur.


In [ ]:
import itkConfig
import itkExtras
itkConfig.ProgressCallback = itkExtras.simple_progress_callback

Start an image processing pipeline with a source, an `ImageFileReader`.

In [ ]:
fileName = 'data/PacMan.png'
reader = itk.ImageFileReader.New(FileName=fileName)

Next, let's create a smoothing filter. To connect the pipeline, specify the `Output` of the reader as the `Input` to the smoother.

In [ ]:
smoother = itk.RecursiveGaussianImageFilter.New(Input=reader.GetOutput())

At this point, no output images have been generated.

We have configured the simple pipeline:

reader -> smoother

In [ ]:
print("reader's Output", reader.GetOutput())
print("smoother's Output", smoother.GetOutput())

To generate the filter outputs, we must call `Update()` on the filter at the end of the pipeline. In this case, it is the smoother.

In [ ]:
smoother.Update()

print("reader's Output", reader.GetOutput())
print("smoother's Output", smoother.GetOutput())

Let's view the images.

In [ ]:
image = reader.GetOutput()
view(image)

In [ ]:
smoothed = smoother.GetOutput()
view(smoothed)

If we call `Update()` on the pipeline, the output pixel data is not needlessly generated because the pipeline is up-to-date.

In [ ]:
smoother.Update()

However, if we change the amount of smoothing, new pixel data does need to be generated from the output of the smoother.

In [ ]:
smoother.SetSigma(3.0)

In [ ]:
view(smoothed)

In [ ]:
smoother.Update()

Note, however, note that the reader does generate its output because it is up-to-date and upstream from the smoother.

If we artificially modify the reader, both the reader and the smoother need to regenerate their outputs.

In [ ]:
reader.Modified()

smoother.Update()

We can stream the pipeline by placing a [StreamingImageFilter](https://itk.org/Doxygen/html/classitk_1_1StreamingImageFilter.html) at the **end** of the pipeline. The smoother generates outputs multiple times, once for each image region streaming division. Since the reader is not capable of streaming, it only generates its output once.

In [ ]:
streamer = itk.StreamingImageFilter.New(Input=smoother.GetOutput())
streamer.SetNumberOfStreamDivisions(3)
reader.Modified()
streamer.Update()

## Exercises

### Exercise 1: The effect of Sigma

Change the value of Sigma on the smoothing filter.

- How is the output effected?
- What are the units of Sigma?

### Exercise 2: Does setting a filter parameter to its current value cause regeneration of its output?

Call `smoother.SetSigma(smoother.GetSigma())` then `smoother.Update()`. 

- Is the output regenerated? 
- Is this expected / desirable?

### Exercise 3: Find other image filtering algorithms

The classes in ITK are organized into **Modules**, and collections of Modules are organized into **Groups**. Examine the [Image Smoothing](https://itk.org/Insight/Doxygen/html/group__ITKSmoothing.html) Module and the [Filtering](https://itk.org/Insight/Doxygen/html/group__Group-Filtering.html) Group. Can any other smoothing or denoising classes be found? 